# Speed Skydiving scoring bulk files processor
Uses the `./data` directory as a data lake.  It finds all valid FlySight files in that directory, from 0 to as many as it finds and fit in memory, and process them all.  Summary of "all files" or "all jumps" at the bottom of the document.

In [2]:
from collections import namedtuple
from copy import deepcopy

from ssscoring.calc import aggregateResults
from ssscoring.calc import convertFlySight2SSScoring
from ssscoring.calc import dropNonSkydiveDataFrom
from ssscoring.calc import getSpeedSkydiveFrom
from ssscoring.calc import isValidJump
from ssscoring.calc import isValidMinimumAltitude
from ssscoring.calc import jumpAnalysisTable
from ssscoring.calc import processAllJumpFiles
from ssscoring.calc import roundedAggregateResults
from ssscoring.calc import totalResultsFrom
from ssscoring.constants import BREAKOFF_ALTITUDE
from ssscoring.constants import FT_IN_M
from ssscoring.constants import PERFORMANCE_WINDOW_LENGTH
from ssscoring.fs1 import getAllSpeedJumpFilesFrom
from ssscoring.fs1 import validFlySightHeaderIn
from ssscoring.notebook import SPEED_COLORS
from ssscoring.notebook import graphAltitude
from ssscoring.notebook import graphAngle
from ssscoring.notebook import graphJumpResult
from ssscoring.notebook import initializeExtraYRanges
from ssscoring.notebook import initializePlot

import csv
import os
import os.path as path

import bokeh.plotting as bp
import bokeh.models as bm
import ipywidgets as widgets
import pandas as pd

In [3]:
DATA_LAKE_ROOT = './data'

---
## <span style="color: yellow">IMPORTANT</span> - Set the drop zone altitude MSL

Set the value in ft (even though SSScoring uses meters) out of convenience because all DZs in the US, many in Asia and Mexico, and many in Europe express the altitude in feet.

### Some DZs

| Drop zone | Alt (ft) |
|-----------|----------|
| Bay Area Skydiving | 23 |
| Skydive Arizona | 1509 |
| Skydive Chicago | 616 |
| Thai Sky Adventures | 21 |

In [4]:
dropZoneAltMSL = 617

In [5]:
dropZoneAltMSLMeters = dropZoneAltMSL/FT_IN_M
display(widgets.HTML('<h2>DZ Altitude = <span style = "color: green">%7.2f ft</span> (%7.2f m)<h1>' % (dropZoneAltMSL, dropZoneAltMSLMeters)))

HTML(value='<h2>DZ Altitude = <span style = "color: green"> 617.00 ft</span> ( 188.06 m)<h1>')

In [6]:
jumpFiles = getAllSpeedJumpFilesFrom(DATA_LAKE_ROOT)

In [7]:
jumpResults = processAllJumpFiles(jumpFiles, altitudeDZMeters = dropZoneAltMSLMeters)

### Air density

**<a href='https://aviex.goflexair.com/blog/density-of-air-air-density' target='_blank'>Pilot's Guide to Air Density</a>** - A useful guide with calculations, lots of information.

![](https://ciurana.eu/personal/air-density-partial-table.png)

---
## Results

In [8]:
aggregate = aggregateResults(jumpResults)
sumResults = totalResultsFrom(aggregate)
display(aggregate)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
17-07-46-08,484.272,187.308,331.380,426.816,477.072,481.860,24.1,486.144
17-09-22-50,485.560174,177.084,328.824,431.532,472.212,485.460,24.2,488.448
17-10-55-36,477.885913,183.384,328.104,423.468,471.888,475.848,24.3,479.376
17-12-28-15,486.2952,177.768,315.504,414.936,472.752,487.584,24.7,488.556
17-13-56-25,472.007455,187.380,334.224,422.352,473.436,456.876,24.3,474.012
18-07-42-08,478.16064,186.120,333.396,425.880,469.440,477.288,24.2,481.176
18-09-22-56,473.271652,177.912,331.092,425.772,466.992,470.556,24.6,476.064
18-11-09-12,487.548,196.236,354.060,444.132,481.608,489.888,23.3,489.888


### Rounded results for training log

In [9]:
roundedResults = roundedAggregateResults(aggregate)

### All jumps

In [10]:
def dumpSimple(table: pd.DataFrame):
    fileName = os.path.join(DATA_LAKE_ROOT, resultRef+'.tsv')
    simpleTable = table.copy()
    simpleTable = simpleTable.drop('speedAngle', axis = 1)
    simpleTable.to_csv(fileName, sep = '\t', index = False)

In [11]:
def displayJumpDataIn(resultsTable: pd.DataFrame):
    table = resultsTable.copy()
    # Experimental
    # For more information on the `interpolate` method and its options, see the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html).
    # Additionally, you can also consider using other libraries like `scipy` which provides more advanced interpolation methods, such as `interp1d` or `griddata`. See the [scipy documentation](https://docs.scipy.org/doc/scipy/reference/interpolate.html) for more information.
    table.vKMh = table.vKMh.apply(round)
    table.hKMh = table.hKMh.apply(round)
    table['altitude (ft)'] = table['altitude (ft)'].apply(round)
    table.netVectorKMh = table.netVectorKMh.apply(round)
    table.index = ['']*len(table)
    display(table)
    # dumpSimple(table)

In [12]:
allJumpsPlot = initializePlot('All jumps in set')
jumpNumber = 0
mixColor = 0
for resultRef in sorted(jumpResults.keys()):
    jumpResult = jumpResults[resultRef]
    if jumpResult.score > 0.0:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - score = %.02f km/h</span></h1>' % (jumpResult.color, resultRef, jumpResult.result, jumpResult.score)        
    else:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s</span></h1>' % (jumpResult.color, resultRef, jumpResult.result)
    display(widgets.HTML(validJumpStatus))

    maxSpeed = jumpResult.maxSpeed
    window = jumpResult.window
    mixColor = (mixColor+1)%len(SPEED_COLORS)
    if jumpResult.score > 0.0:
        display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), end scoring window at %d m (%d ft)'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
        if not isValidMinimumAltitude(jumpResult.data.altitudeAGL.max()):
            display(widgets.HTML('<span style="color: yellow"><span style="font-weight: bold">Warning:</span> exit altitude ASL was lower than the minimum scoring altitude according to IPC and USPA.'))
        display(widgets.HTML('</h3>'))
        # TODO: Move this to an in-notebook function or module.
        displayJumpDataIn(jumpResult.table)
        individualPlot = initializePlot(resultRef)
        individualPlot = initializeExtraYRanges(individualPlot, startY = min(jumpResult.data.altitudeAGLFt)-500.0, endY = max(jumpResult.data.altitudeAGLFt)+500.0)
        graphAltitude(individualPlot, jumpResult)
        graphAngle(individualPlot, jumpResult)
        hoverValue = bm.HoverTool(tooltips = [ ('Y-val', '@y{0.00}'), ])
        individualPlot.add_tools(hoverValue)
        graphJumpResult(individualPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[0])
        graphJumpResult(allJumpsPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[mixColor],
                        legend = '%s - %.2f' % (resultRef, jumpResult.score),
                        showIt = False)

HTML(value='<hr><h1><span style="color: #0f0">17-07-46-08 jump - 🟢 valid - score = 484.27 km/h</span></h1>')

HTML(value='<h3>Max speed = 486; exit at 4196 m (13766 ft), end scoring window at 1940 m (6365 ft)')

HTML(value='</h3>')

,time,vKMh,hKMh,speedAngle,distanceFromExit,altitude (ft),netVectorKMh
,5.0,187,148,51.7,237.700000,13296,239
,10.0,331,105,72.4,408.000000,12070,348
,15.0,427,68,80.9,523.800000,10304,432
,20.0,477,21,87.5,572.000000,8218,478
,25.0,482,15,88.2,569.280586,6406,482


HTML(value='<hr><h1><span style="color: #0f0">17-09-22-50 jump - 🟢 valid - score = 485.56 km/h</span></h1>')

...

,time,vKMh,hKMh,speedAngle,distanceFromExit,altitude (ft),netVectorKMh
,5.0,178,157,48.5,254.400000,12871,237
,10.0,331,106,72.3,441.100000,11655,348
,15.0,426,43,84.2,540.000000,9908,428
,20.0,467,33,86.0,547.000000,7895,468
,25.0,471,60,82.8,489.560356,5915,474


HTML(value='<hr><h1><span style="color: #0f0">18-11-09-12 jump - 🟢 valid - score = 487.55 km/h</span></h1>')

HTML(value='<h3>Max speed = 490; exit at 4134 m (13564 ft), end scoring window at 1878 m (6162 ft)')

HTML(value='</h3>')

,time,vKMh,hKMh,speedAngle,distanceFromExit,altitude (ft),netVectorKMh
,5.0,196,153,52.1,228.800000,13074,249
,10.0,354,105,73.4,399.500000,11700,369
,15.0,444,46,84.1,502.000000,9814,446
,20.0,482,9,88.9,532.200000,7669,482
,25.0,490,25,87.1,512.649492,6194,491


---
## Summary of all speed skydives in the data lake

**<a href='https://intimescoring.com/Results/EventResults?dbid=100&mid=1074&smid=0&mdcid=1273&dseid=0' target='_blank'>InTime Scoring - 2024 USPA Speed Open results</a>**

In [13]:
display(roundedResults)
display(sumResults)
bp.show(allJumpsPlot)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
17-07-46-08,484,187,331,427,477,482,24.1,486
17-09-22-50,486,177,329,432,472,485,24.2,488
17-10-55-36,478,183,328,423,472,476,24.3,479
17-12-28-15,486,178,316,415,473,488,24.7,489
17-13-56-25,472,187,334,422,473,457,24.3,474
18-07-42-08,478,186,333,426,469,477,24.2,481
18-09-22-56,473,178,331,426,467,471,24.6,476
18-11-09-12,488,196,354,444,482,490,23.3,490


,totalSpeed,meanSpeed,maxScore
totalSpeed,3845.001034,480.625129,487.548
